In [ ]:
import requests, os
from tqdm.notebook import tqdm as tq

# https://yandex.ru/dev/disk/api/reference/public.html - апи публичных ссылок
class YaPubLink():

    def __init__(self, public_key):
        self._public_key = public_key

    def get_meta(self, path='/', **kwargs):
        params = {'public_key': self._public_key, 'path': path}
        params.update(kwargs)
        res = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources', params=params)
        if res.status_code == 200:
            j_obj = res.json()
            return j_obj
        else:
            print('Ошибка:', f'{res.status_code = } {res.text = }')

    def get_download_link(self, path='/', **kwargs):
        params = {'public_key': self._public_key, 'path': path}
        params.update(kwargs)
        res = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download', params=params)
        if res.status_code == 200:
            j_obj = res.json()
            return j_obj
        else:
            print('Ошибка:', f'{res.status_code = } {res.text = }')

    def recursive_tree(self, tree=None, path='/'):
        if tree is None:
            tree = {}
        meta = self.get_meta(path)
        tree['name'] = meta['name']
        tree['path'] = meta['path']
        tree['type'] = meta['type']
        tree['size'] = meta.get('size', None)
        if meta['type'] == 'dir':
            _embedded = meta['_embedded']
            if _embedded['total'] > _embedded['limit']:
                meta = self.get_meta(path, limit=_embedded['total'])
                _embedded = meta['_embedded']
            tree['items'] = []
            for item in _embedded['items']:
                leaf = {}
                tree['items'].append(leaf)
                if item['type'] == 'dir':
                    self.recursive_tree(leaf, item['path'])
                else:
                    leaf['name'] = item['name']
                    leaf['path'] = item['path']
                    leaf['type'] = item['type']
                    leaf['size'] = item['size']
        return tree

    def recursive_download(self, tree, target_dir, to_download_pred, files_list=None):

        def sizeof_unit(num):
            for unit in ['bytes', 'KB', 'MB', 'GB']:
                if num < 1024.0:
                    return num, unit
                num /= 1024.0
            return num, 'TB'

        def sizeof_fmt(num):
            size_unit = sizeof_unit(num)
            return f'{size_unit[0]:.1f} {size_unit[1]}'

        def get_file(download_url, file_name, file_path):
            res_d = requests.get(download_url, stream=True)
            f_size = int(res_d.headers.get('content-length', 0))
            desc = f'{file_name}: {sizeof_fmt(f_size)}'
            chunk_size = 1024 * 1024
            num_bars = (f_size + chunk_size - 1)// chunk_size
            new_filename = os.path.join(file_path, file_name)
            with open(new_filename, 'wb') as fp:
                for data in tq(res_d.iter_content(chunk_size), desc=desc, total=num_bars, unit='MB'):
                    fp.write(data)
            return new_filename

        if files_list is None:
            files_list = []
        if tree['type'] == 'file':
            if to_download_pred(tree['name'], tree['size']):
                os.makedirs(target_dir, exist_ok=True)
                download_url = self.get_download_link(tree['path'])['href']
                res_file = get_file(download_url, tree['name'], target_dir)
                files_list.append(res_file)
        else:
            target_dir = os.path.join(target_dir, tree['path'][1:])
            for item in tree['items']:
                self.recursive_download(item, target_dir, to_download_pred, files_list)
        return files_list


In [ ]:
# каталог для загрузки файлов
TARGET_DIR = '/content/original_videos'

# экземпляр класса YaPubLink на основе публичной ссылки
ypl = YaPubLink('https://disk.yandex.ru/d/Ofp593YruLRYsQ')

# функция предиктор, загружать ли файл по условию
# например расширение == 'avi' и размер <= 512 МБ
def to_download(filename, filesize):
    if filename.split('.')[-1] == 'avi' and filesize <= 512 * 1024 * 1024:
        return True
    return False

# структура файлов на я-диске
tree = ypl.recursive_tree()

# подготовка каталога для загрузки файлов
!rm -rf {TARGET_DIR}
os.makedirs(TARGET_DIR, exist_ok=True)

# рекурсивное скачивание файлов из созданной структуры
# с отбором по условию в функции-предикторе
downloaded_files = ypl.recursive_download(tree, TARGET_DIR, to_download_pred=to_download)

ГС-ЦОД-9 2023-09-08 15-16-44_475.avi: 92.4 MB:   0%|          | 0/93 [00:00<?, ?MB/s]

ГС-ЦОД-10 2023-09-15 13-50-31_576.avi: 227.6 MB:   0%|          | 0/228 [00:00<?, ?MB/s]

ЦОД-12 2023-09-15 12-56-15_704.avi: 319.2 MB:   0%|          | 0/320 [00:00<?, ?MB/s]

In [ ]:
# просмотр списка скачанных файлов
downloaded_files

['/content/original_videos/Без касок/ГС-ЦОД-9 2023-09-08 15-16-44_475.avi',
 '/content/original_videos/В касках/ГС-ЦОД-10 2023-09-15 13-50-31_576.avi',
 '/content/original_videos/В касках/ЦОД-12 2023-09-15 12-56-15_704.avi']

In [ ]:
# просмотр структуры файлов на я-диске
tree

{'name': 'Видео цех',
 'path': '/',
 'type': 'dir',
 'size': None,
 'items': [{'name': 'Без касок',
   'path': '/Без касок',
   'type': 'dir',
   'size': None,
   'items': [{'name': 'ГС-ЦОД-10 2023-09-04 09-00-38_734.avi',
     'path': '/Без касок/ГС-ЦОД-10 2023-09-04 09-00-38_734.avi',
     'type': 'file',
     'size': 2101649408},
    {'name': 'ГС-ЦОД-10 2023-09-06 11-45-11_101.avi',
     'path': '/Без касок/ГС-ЦОД-10 2023-09-06 11-45-11_101.avi',
     'type': 'file',
     'size': 2101165056},
    {'name': 'ГС-ЦОД-10 2023-09-07 15-11-56_329.avi',
     'path': '/Без касок/ГС-ЦОД-10 2023-09-07 15-11-56_329.avi',
     'type': 'file',
     'size': 1495284224},
    {'name': 'ГС-ЦОД-8 2023-09-04 07-05-48_170.avi',
     'path': '/Без касок/ГС-ЦОД-8 2023-09-04 07-05-48_170.avi',
     'type': 'file',
     'size': 2092208128},
    {'name': 'ГС-ЦОД-8 2023-09-04 21-25-37_232.avi',
     'path': '/Без касок/ГС-ЦОД-8 2023-09-04 21-25-37_232.avi',
     'type': 'file',
     'size': 2092515840},
    {